In [2]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2532688127768558292]

In [4]:
import platform
platform.platform()

'Windows-10-10.0.18362-SP0'

In [1]:
import os

dataset_path = "./voxceleb/dev_wav"

print(sorted(os.listdir(dataset_path))[:5])
print(len(sorted(os.listdir(dataset_path)))) # # of speakers

['id10001', 'id10002', 'id10003', 'id10004', 'id10005']
1211


In [17]:
a = 0
for (root, dirs, files) in os.walk(dataset_path):
    a += len(files)

In [18]:
# # of utterances
a

148642

In [20]:
from scipy.io import wavfile

b, c = 0, 0
for (root, dirs, files) in os.walk(dataset_path):
    for filename in files:
        _, samples = wavfile.read(os.path.join(root, filename))
        
        b = max(b, len(samples))
        c = min(c, len(samples))

In [21]:
# maximum, minimum length of files
b, c

(2318721, 0)

In [2]:
data_length = 3 ** 9

In [4]:
from scipy.io import wavfile

save_npy_path = "./npzs"
if not os.path.exists(save_npy_path):
    os.mkdir(save_npy_path)

for root_dir in os.listdir(dataset_path): # id10001 ~ id11211 (1211 speakers)   
    
    utts = 0 # num of wav files for each speakers
    for (root, dirs, files) in os.walk(os.path.join(dataset_path, root_dir)):
        utts += len(files)
    
    i = 0
    data = np.zeros((utts, data_length))

    for (root, dirs, files) in os.walk(os.path.join(dataset_path, root_dir)):
        for filename in files:
            
            _, samples = wavfile.read(os.path.join(root, filename))
            
            try: 
                if len(samples) < data_length:
                    raise Exception("length of samples < data_length = 3 ** 9")
            except Exception as e:
                print(e)
            
            start = int(len(samples) - data_length) // 2
            end = start + data_length
            
            data[i] = samples[start:end]            
            i = i + 1
    
    np.savez_compressed(os.path.join(save_npy_path, "{}.npz".format(root_dir)), data)

In [51]:
data.shape

(61, 19683)

In [5]:
dataset_path = "./voxceleb123/test_wav"
data_length = 3 ** 9

In [13]:
from scipy.io import wavfile
import os

save_npy_path = "./Voxceleb/test"
if not os.path.exists(save_npy_path):
    os.mkdir(save_npy_path)

for root_dir in os.listdir(dataset_path): # id10001 ~ id11211 (1211 speakers)

    for (root, dirs, files) in os.walk(os.path.join(dataset_path, root_dir)):
        
        if len(files) == 0: continue
        
        utts = len(files)            
        data = np.zeros((utts, data_length))
        
        for i, filename in enumerate(files):
            
            _, samples = wavfile.read(os.path.join(root, filename))
            
            try: 
                if len(samples) < data_length:
                    raise Exception("length of samples < data_length = 3 ** 9")
            except Exception as e:
                print(e)
            
            start = int(len(samples) - data_length) // 2
            end = start + data_length
            
            data[i] = samples[start:end]            

        # to fit the requirements of validation format
        # http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1.html
        # http://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test.txt
        np.savez_compressed(os.path.join(save_npy_path,
                                         "{}_{}".format(root_dir, root.split(os.path.sep)[-1])), data)